In [7]:

import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession
import numpy as np
from fake_useragent import UserAgent
from urllib.request import Request, urlopen
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.data import url2pathname

from nltk.tokenize import word_tokenize
from nltk.stem.porter import *

from bs4 import BeautifulSoup

from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english')) 
stop_words.update(['le', 'eu', 'span', 'ago', 'pp', 'ue', 'div', 'src', 'page', 'egp', 'url', 'cdn', 'alt', 'com', 'net', 'org', 'cdn', 'img', 'google','eg','usd','http','https','net','us','eg'])

import time
t0=time.time()


import mysql.connector

# Connect to the database
mysql = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="seopro"
)

# Create a cursor object
mycursor = mysql.cursor()

# Execute an SQL query to select the last row from the table
mycursor.execute("SELECT uid, niche FROM given_niche ORDER BY id DESC LIMIT 1")

# Fetch the last row from the table
niche_arr = mycursor.fetchone()

# # Convert niche[0] to a string
# niche= str(niche_arr[0])

# # Print the last row as a string
# print(niche)

# Extract the id and niche values from the row
uid = niche_arr[0]
niche = niche_arr[1]

# Print the niche and id values
print(niche)
print(uid)


def get_text(url):
    try:
        req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req,timeout=5).read()
        soup = BeautifulSoup(webpage, "html.parser")
        texts = soup.findAll(text=True)
        res=u" ".join(t.strip() for t in texts if t.parent.name not in ['style', 'script', 'head', 'title', 'meta', '[document]'])
        return(res)
    except:
        return False
    
"""Step 2: Get the URLs from competitors

"""

def get_source(url):
    try:
        session = HTMLSession()
        response = session.get(url)

        return response
    except requests.exceptions.RequestException as e:
        print(e)
def scrape_google(query):

    query = urllib.parse.quote_plus(query)
    response = get_source("https://www.google.com/search?q=" + query)
    links = list(response.html.absolute_links)
    # google_domains = ('https://www.google.', 
    #                   'https://google.', 
    #                   'https://webcache.googleusercontent.', 
    #                   'http://webcache.googleusercontent.', 
    #                   'https://policies.google.',
    #                   'https://support.google.',
    #                   'https://maps.google.')
    
    google_domains = ('https://www.google.', 
                      'https://google.')
     

    for url in links[:]:
        if url.startswith(google_domains):
            links.remove(url)

    return links

# niche = input("enter your niche")
# scrape_google(niche)

"""Step 3: Analyse the text and get the most important words."""

# niche = input("enter your niche: ")
links=scrape_google(niche)
print(links)
text=[]
for i in links:
  t=get_text(i)
  if t:
    text.append(t)

    word_tokens = word_tokenize(t)

   
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    


p_stemmer = PorterStemmer()




for word in filtered_sentence:
    # if len(word) >= 3:
    word+' --> '+p_stemmer.stem(word)

from nltk.stem.snowball import SnowballStemmer
s_stemmer = SnowballStemmer(language='english')



for word in filtered_sentence:
    if len(word) >=4:
        word+' --> '+s_stemmer.stem(word)
u=p_stemmer.stem(word)
import mysql.connector

########
# Modify the tf_idf_analysis function to return a DataFrame
def tf_idf_analysis(keyword):
    v = TfidfVectorizer(min_df=1,analyzer='word',ngram_range=(1,2),stop_words=list(stop_words))
    x = v.fit_transform(text)
    f = pd.DataFrame(x.toarray(), columns = v.get_feature_names_out())
    d=pd.concat([pd.DataFrame(f.mean(axis=0)),pd.DataFrame(f.max(axis=0))],axis=1)
    tf=pd.DataFrame((f>0).sum(axis=0))
    d=d.reset_index().merge(tf.reset_index(),on='index',how='left')
    d.columns=['word','average_tfidf','max_tfidf','frequency']
    d['frequency']=round((d['frequency']/len(text))*100)
    d['max_tfidf'] = d['max_tfidf'].round(3)
    d['average_tfidf'] = d['average_tfidf'].round(3)

    d= d[d['word'].str.isalpha()].sort_values('frequency',ascending=False).head(35)
    
    return d


# Call the tf_idf_analysis function and store the output in a DataFrame variable
output_df = tf_idf_analysis(u)
print (output_df)
# Convert the DataFrame to a dictionary
output_dict = output_df.to_dict('records')

# Connect to the database
mysql = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="",
    database="seopro"
)

# Insert the data into the database
mycursor = mysql.cursor()
sql = "INSERT INTO keywords (word, average_tfidf, max_tfidf, frequency, niche, uid) VALUES (%s, %s, %s, %s, %s, %s)"
for row in output_dict:
    val = (row['word'], row['average_tfidf'], row['max_tfidf'], row['frequency'], niche, uid)
    mycursor.execute(sql, val)
mysql.commit()

# Print a message to confirm that the data has been saved
print("Data has been saved to the database.")



#try this if above is not working

# import requests
# import urllib
# import pandas as pd
# from requests_html import HTML
# from requests_html import HTMLSession
# import numpy as np
# from fake_useragent import UserAgent
# from urllib.request import Request, urlopen
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import CountVectorizer
# from nltk.data import url2pathname

# from nltk.tokenize import word_tokenize
# from nltk.stem.porter import *

# from bs4 import BeautifulSoup

# from nltk.corpus import stopwords

# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

# stop_words = set(stopwords.words('english')) 
# stop_words.update(['le', 'eu', 'span', 'ago', 'pp', 'ue', 'div', 'src', 'page', 'egp', 'url', 'cdn', 'alt', 'com', 'net', 'org', 'cdn', 'img', 'google','eg','usd','http','https','net','us','eg'])




# import mysql.connector

# # Connect to the database
# mysql = mysql.connector.connect(
#     host="localhost",
#     user="root",
#     password="",
#     database="seopro"
# )

# # Create a cursor object
# mycursor = mysql.cursor()

# # Execute an SQL query to select the last row from the table
# mycursor.execute("SELECT uid, niche FROM given_niche ORDER BY id DESC LIMIT 1")

# Fetch the last row from the table
# niche_arr = mycursor.fetchone()

# # # Convert niche[0] to a string
# # niche= str(niche_arr[0])

# # # Print the last row as a string
# # print(niche)

# # Extract the id and niche values from the row
# uid = niche_arr[0]
# niche = niche_arr[1]

# # Print the niche and id values
# print(niche)
# print(uid)


# def get_text(url):
#     try:
#         req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
#         webpage = urlopen(req,timeout=5).read()
#         soup = BeautifulSoup(webpage, "html.parser")
#         texts = soup.findAll(text=True)
#         res=u" ".join(t.strip() for t in texts if t.parent.name not in ['style', 'script', 'head', 'title', 'meta', '[document]'])
#         return(res)
#     except:
#         return False
    
# """Step 2: Get the URLs from competitors

# """

# def get_source(url):
#     try:
#         session = HTMLSession()
#         response = session.get(url)

#         return response
#     except requests.exceptions.RequestException as e:
#         print(e)
# def scrape_google(query):

#     query = urllib.parse.quote_plus(query)
#     response = get_source("https://www.google.com/search?q=" + query)
#     links = list(response.html.absolute_links)
#     # google_domains = ('https://www.google.', 
#     #                   'https://google.', 
#     #                   'https://webcache.googleusercontent.', 
#     #                   'http://webcache.googleusercontent.', 
#     #                   'https://policies.google.',
#     #                   'https://support.google.',
#     #                   'https://maps.google.')
    
#     google_domains = ('https://www.google.', 
#                       'https://google.')
     

#     for url in links[:]:
#         if url.startswith(google_domains):
#             links.remove(url)

#     return links

# # niche = input("enter your niche")
# # scrape_google(niche)

# """Step 3: Analyse the text and get the most important words."""

# # niche = input("enter your niche: ")
# links=scrape_google(niche)
# print(links)
# text=[]
# for i in links:
#   t=get_text(i)
#   if t:
#     text.append(t)

#     word_tokens = word_tokenize(t)

   
#     filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    


# p_stemmer = PorterStemmer()




# for word in filtered_sentence:
#     # if len(word) >= 3:
#     word+' --> '+p_stemmer.stem(word)

# from nltk.stem.snowball import SnowballStemmer
# s_stemmer = SnowballStemmer(language='english')



# for word in filtered_sentence:
#     if len(word) >=4:
#         word+' --> '+s_stemmer.stem(word)
# u=p_stemmer.stem(word)
# import mysql.connector

# ########
# # Modify the tf_idf_analysis function to return a DataFrame
# def tf_idf_analysis(keyword):
#     v = TfidfVectorizer(min_df=1,analyzer='word',ngram_range=(1,2),stop_words=list(stop_words))
#     x = v.fit_transform(text)
#     f = pd.DataFrame(x.toarray(), columns = v.get_feature_names_out())
#     d=pd.concat([pd.DataFrame(f.mean(axis=0)),pd.DataFrame(f.max(axis=0))],axis=1)
#     tf=pd.DataFrame((f>0).sum(axis=0))
#     d=d.reset_index().merge(tf.reset_index(),on='index',how='left')
#     d.columns=['word','average_tfidf','max_tfidf','frequency']
#     d['frequency']=round((d['frequency']/len(text))*100)
#     d['max_tfidf'] = d['max_tfidf'].round(3)
#     d['average_tfidf'] = d['average_tfidf'].round(3)

#     d= d[d['word'].str.isalpha()].sort_values('frequency',ascending=False).head(35)
    
#     return d


# # Call the tf_idf_analysis function and store the output in a DataFrame variable
# output_df = tf_idf_analysis(u)
# print (output_df)
# # Convert the DataFrame to a dictionary
# output_dict = output_df.to_dict('records')

# # Connect to the database
# mysql = mysql.connector.connect(
#     host="localhost",
#     user="root",
#     passwd="",
#     database="seopro"
# )

# # Insert the data into the database
# mycursor = mysql.cursor()
# sql = "INSERT INTO keywords (word, average_tfidf, max_tfidf, frequency, niche, uid) VALUES (%s, %s, %s, %s, %s, %s)"
# for row in output_dict:
#     val = (row['word'], row['average_tfidf'], row['max_tfidf'], row['frequency'], niche, uid)
#     mycursor.execute(sql, val)
# mysql.commit()

# # Print a message to confirm that the data has been saved
# print("Data has been saved to the database.")


print(time.time()-t0)





[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


dogs
9
['https://maps.google.com/maps?output=search&q=dogs&entry=mc&sa=X&ved=2ahUKEwiIhZuH_cX_AhUcpVYBHbfcD_oQ0pQJegQIBBAB', 'https://translate.google.com/translate?hl=ar&sl=en&u=https://www.britannica.com/animal/dog&prev=search&pto=aue', 'https://www.youtube.com/watch?v=IACqfc8BtK4', 'https://en.wikipedia.org/wiki/Lists_of_dogs', 'https://translate.google.com/translate?hl=ar&sl=en&u=https://a-z-animals.com/pets/dogs/&prev=search&pto=aue', 'https://translate.google.com/translate?hl=ar&sl=en&u=https://www.cdc.gov/healthypets/pets/dogs.html&prev=search&pto=aue', 'https://www.goodhousekeeping.com/life/pets/g4531/cutest-dog-breeds/', 'https://en.wikipedia.org/wiki/Pleistocene_wolf', 'https://www.dogstrust.org.uk/', 'https://translate.google.com/translate?hl=ar&sl=en&u=https://en.wikipedia.org/wiki/Dog&prev=search&pto=aue', 'https://ar.wikipedia.org/wiki/%D9%83%D9%84%D8%A8', 'https://translate.google.com/translate?hl=ar&sl=en&u=https://www.rspca.org.uk/adviceandwelfare/pets/dogs&prev=search

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_5784\118540272.py:70: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  texts = soup.findAll(text=True)


              word  average_tfidf  max_tfidf  frequency
20082         dogs          0.140      0.498       74.0
23640       family          0.011      0.033       74.0
19214          dog          0.162      0.420       74.0
49529       search          0.006      0.027       71.0
22196      english          0.009      0.046       71.0
12032       breeds          0.039      0.159       68.0
16242      content          0.008      0.039       68.0
7925        animal          0.016      0.090       68.0
41168          one          0.018      0.061       68.0
30927  information          0.008      0.052       65.0
45281      privacy          0.006      0.036       65.0
28712         home          0.010      0.087       65.0
10215       become          0.006      0.038       61.0
37438          may          0.017      0.081       61.0
49637          see          0.010      0.035       61.0
56846       united          0.005      0.012       61.0
35658       living          0.005      0.027    